# 新一代Boost框架的使用

在[2.10 提升方法](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/杭电机器学习课程及代码/2.10%20提升方法.ipynb)中我们简单提到过，在SKLearn之外还有很多优秀的，基于Boosting Tree方法的衍生框架。包括XGBoost，LightGBM等，而各种开源比赛方案对它们的应用，也从事实上证明了这些框架的优秀性能。

因此这一节会简要介绍一些新框架的原理和用法。

参考文章：

- [XGBoost PPT](https://homes.cs.washington.edu/~tqchen/pdf/BoostedTree.pdf)
- [XGBoost: A Scalable Tree Boosting System](https://arxiv.org/abs/1603.02754)
- [Introduction to Boosted Trees](https://xgboost.readthedocs.io/en/latest/tutorials/model.html)
- [XGBoost原理](https://www.zhihu.com/question/58883125/answer/206813653)

## XGBoost

### 简介

**XGBoost** 全称为“eXtreme Gradient Boosting”，是一种基于决策树的集成机器学习算法，使用梯度提升框架，适用于分类和回归问题。XGBoost 在各种比赛及比赛平台中大放异彩，比如Kaggle，天池，DataCastle等等，可以说是所有选手的必备武器之一。XGBoost 的项目主页参见[这里](https://xgboost.ai)。

XGBoost 最初由陈天奇开发。陈天奇本科毕业于上海交通大学 ACM 班，博士毕业于华盛顿大学计算机系，研究方向为大规模机器学习，2020年他将加入 CMU 出任助理教授。有兴趣的可以看看他的文章[《陈天奇：机器学习科研的十年》](http://www.sohu.com/a/328234576_129720)以及[访谈](https://cosx.org/2015/06/interview-of-tianqi)，也可以看看他的[知乎主页](https://www.zhihu.com/people/crowowrk/activities)。

下图展示了从决策树到 XGBoost 算法的发展过程：

![img](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/xgboost%E5%8F%91%E5%B1%95%E5%8E%86%E7%A8%8B.jpeg?raw=true)

图片刷不出来的请看[这里](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/xgboost发展历程.jpeg)

在 [XGBoost.ai](https://xgboost.ai) 官网有对 XGBoost原理有通俗易懂的解释。

现在简要介绍如下，其中的各种符号跟论文保持一致，所以和之前的 GBDT 介绍可能有少许差别。

### 目标函数：经验损失 + 正则化

训练模型的目标是找到最好的参数 $\theta$ ，能够最佳拟合训练数据 $x_i$ 和标签 $y_i$。所以XGBoost为了训练模型，定义了一个**目标函数（objective function）** 来告诉模型如何去拟合训练数据。

目标函数由两部分组成：**经验损失**（又叫**训练损失**，**training loss**）和**正则项（regularization term）**。

$$\large obj(\theta)=L(\theta)+\Omega(\theta)$$

其中， $L$ 就是训练损失，而 $\Omega$ 就是正则项。

通常，$L$ 会选择均方误差，如下：

$$\large L(\theta)=\sum_i(y_i-\hat y_i)^2$$

其中，$\hat y_i=\sum_j \theta_j x_{ij}$，也就是模型的预测值。

另一种常见的损失函数叫做**logistic loss**，也就是之前在逻辑回归中讲过的损失函数：

$$\large L(\theta) = \sum_i \left[ y_i \ln(1+e^{-\hat y_i})+(1-y_i)\ln(1+e^{-\hat y_i}) \right]$$

至于正则项$\Omega$，就是控制模型复杂度的，让模型不至于过拟合，以前的章节讲过多次，这里不再赘述。

### 决策树的集成

XGBoost模型是基于决策树的集成（**decision tree ensembles**）而来，这个集成模型是由一组**分类与回归树（CART）**所构成的，CART 的分裂基于 gini 系数，关于 CART 的详细描述可以参见《统计学习方法》中决策树一章。

这里给了一个简单的例子，比如我们要运用 CART 来判断某人是否喜欢电脑游戏。我们输入一家人的年龄、性别、职业等信息，然后得到一颗决策树：

![img](https://raw.githubusercontent.com/dmlc/web-data/master/xgboost/model/cart.png)

我们将家庭成员分为不同的叶子结点（leaves），然后在相应的叶子上给他们分配分数（score）。

>score 怎么来的？

### 树的Boosting



回顾一下**泰勒公式**：

> 设函数 $f(x)$ 在包含 $x_0$ 的某区间 $(a,b)$ 内有 $n+1$ 阶导数，则对任一 $x\in(a,b)$，有
>
> $$\large f(x)=\frac{f(x_0)}{0!} + \frac{f'(x_0)}{1!}(x-x_0) + \frac{f''(x_0)}{2!}(x-x_0)^2 + \cdots + \frac{f^{(n)}(x_0)}{n!}(x-x_0)^n+R_n(x)$$
>
>其中，$R_n(x)$ 是泰勒公式的余项。

### 算法回顾

我们从上一小节的图片中得知，XGBoost 是由 **GBDT** 发展而来。

首先回顾一下之前的知识：

**Boosting**方法，它假设我们的分类器是一系列基本分类器的线性组合：

$$\large f(x)=\sum^M_{m=1}\alpha_mG_m(x)$$

这里 $\alpha_m$ 就是基本分类器的权重系数，而 $G_m(x)$ 就是基本分类器。

而**提升树模型**就可以表示为决策树的加法模型：

$$\large f_M(x)=\sum_{m=1}^MT(x;\Theta)$$

其中，$T(x_i;\Theta_m)$ 表示第 $m$ 棵决策树， $\Theta_m$ 为决策树的参数，$M$ 为树的个数。

提升树算法采用**前向分步算法**。首先确定初始提升树 $f_0(x)=0$ ，这样一步一步迭代计算，第 $m$ 步的模型是

$$\large f_m(x)=f_{m-1}(x)+T(x;\Theta_m)$$

如果我们采用**梯度提升（Gradient Boosting）**方法求解最优参数，就构成了梯度提升树模型。

已知梯度提升树的损失函数为 $L(y,f(x))$，为了让它最小，只要让每一个训练样本的损失最小，所以我们求损失函数的负梯度在当前模型的值，作为残差 $r_{mi}$ 的近似值，来拟合回归树：

$$\large r_{mi}=-\left[ \frac{\partial L(y,f(x_i))}{\partial f(x_i)} \right]_{f(x)=f_{m-1}(x)}=y-f(x_i)$$